# Отток клиентов

**Данные:** исторические данные о поведении клиентов и расторжении договоров с банком. 

**Описание данных:**

Признаки:
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

Целевой признак:
- Exited — факт ухода клиента

**Цель исследования:** спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Построить модель с предельно большим значением *F1*-меры. Нужно довести метрику до 0.59.

**Ход исследования:**
Данные о поведении клиентов находятся в файле Churn.csv. Так как о качестве данных мне неизвестно, сначала я проведу предобработку. Потом я разобью данные на 3 выборки: тренировочную, валидационную и тестовую, посмотрю на дисбаланс классов и предприму меры для решения проблемы. Также проверю разные модели, рассчитаю F1-меру и AUC-ROC, выберу лучшую модель. В конце лучшую модель проверю на тестовой выборке и посмотрю на адекватность.

**Исследование пройдёт в несколько этапов:**

- Импорт библиотек.
- Общая информация о данных.
- Предобработка.
- Разбивка данных на выборки.
- Исследование моделей.
- Проверка лучшей модели на тестовой выборке.
- Проверка модели на адекватность.
- Вывод.

## Импорт библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score

## Подготовка данных

### Загрузка данных и просмотр информации о них

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure             float64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [4]:
data.shape

(10000, 14)

**Промежуточный вывод:**
- названия колонок надо изменить
- проверить данные на пропуски и заполнить их, если будут
- проверить данные на дубликаты
- некоторые типы данных надо заменить на bool

### Изменение названий колонок

Названия столбцов приведу к нижнему регистру, добавлю змеиный регистр к тем столбцам, где несколько слов:

In [5]:
data.columns = map(str.lower, data.columns)

data = data.rename(columns={'rownumber': 'row_number', 'customerid': 'customer_id', 'creditscore': 'credit_score', \
                            'numofproducts': 'num_of_products', 'hascrcard': 'has_cr_card', \
                            'isactivemember': 'is_active_member', 'estimatedsalary': 'estimated_salary'})

### Проверка на пропуски и их заполнение

Проверю, есть ли пропуски:

In [6]:
data.isnull().sum()

row_number            0
customer_id           0
surname               0
credit_score          0
geography             0
gender                0
age                   0
tenure              909
balance               0
num_of_products       0
has_cr_card           0
is_active_member      0
estimated_salary      0
exited                0
dtype: int64

Так как в колонке tenure пропусков меньше 10% от данных, проще их удалить, так как заполнение медианой по другому признаку поменяет распределение, я проверяла.

In [7]:
data = data.dropna()

data['tenure'].isna().sum() #проверка

0

Еще одна проверка:

In [8]:
data.shape

(9091, 14)

Теперь пропусков в данных нет.

### Проверка на дубликаты

In [9]:
data.duplicated().sum() #Проверила на дубликаты

0

Дубликатов нет.

### Замена типов данных

Эти данные я изменю на формат bool, так как потом буду проводить стандартизацию. Чтобы их не нужно было стандартизировать.

- has_cr_card            int64
- is_active_member       int64

In [10]:
data[['has_cr_card', 'is_active_member']] = data[['has_cr_card', 'is_active_member']].astype('bool')

In [11]:
data[['has_cr_card', 'is_active_member']].dtypes #проверка

has_cr_card         bool
is_active_member    bool
dtype: object

**Вывод:**
- удалены пропуски
- изменены названия колонок
- некоторым колонкам присвоен новый тип данных
- проведена проверка на дубли

## Исследование задачи

### Прямое кодирование и разбивка на выборки

Техникой OHE категориальные признаки переведу в численные, также удалю первые столбцы, чтобы не угодить в dummy-ловушку.

Разобью данные на тренировочную и валидационную выборку, а потом валидационную разобью пополам. Так я получу 3 выборки с соотношением:
- тренировочная выборка 0.6
- валидационная выборка 0.2
- тестовая выборка 0.2

Также я решила удалить некоторые столбцы, которые к таргету не имеют отношения и не помогут при построении модели: 'customer_id', 'row_number', 'surname'.

In [12]:
data = data.drop(['customer_id', 'row_number', 'surname'], axis='columns')

data_ohe = pd.get_dummies(data, drop_first=True)

target = data_ohe['exited']
features = data_ohe.drop('exited', axis='columns')

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print(features_train.shape, features_valid.shape, features_test.shape) #проверка размера выборок
print(target_train.shape, target_valid.shape, target_test.shape)

(5454, 11) (1818, 11) (1819, 11)
(5454,) (1818,) (1819,)


### Стандартизация данных

Чтобы все численные признаки были одинаково значимыми, масштабирую их:

In [13]:
features_train.dtypes

credit_score           int64
age                    int64
tenure               float64
balance              float64
num_of_products        int64
has_cr_card             bool
is_active_member        bool
estimated_salary     float64
geography_Germany      uint8
geography_Spain        uint8
gender_Male            uint8
dtype: object

Буду стандартизировать: 
- 'credit_score', 
- 'age', 
- 'tenure', 
- 'balance', 
- 'num_of_products', 
- 'estimated_salary'.

In [14]:
pd.options.mode.chained_assignment = None
numeric = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary', 'num_of_products']

scaler = StandardScaler()

features_train[numeric] = scaler.fit_transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Модель без учета баланса классов

Проверю, как отработает модель без учета проверки баланса классов:

In [15]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predicted_valid = pd.Series(model.predict(features_valid))

print("Accuracy:", accuracy_score(target_valid, predicted_valid))
print("F1-мера:", f1_score(target_valid,predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

Accuracy: 0.7904290429042904
F1-мера: 0.4926764314247669
AUC-ROC: 0.6797786314764468


Баланс классов в target_train:

In [16]:
target_train.value_counts(normalize=True)

0    0.793546
1    0.206454
Name: exited, dtype: float64

Также проверю распределение классов в результатах предсказаний решающего дерева:

In [17]:
predicted_valid.value_counts(normalize=True)

0    0.794279
1    0.205721
dtype: float64

И построю константную модель, которая будет присваивать нули:

In [18]:
target_train_pred_constant = pd.Series(pd.Series(0, index=target_train.index))

print("Accuracy константной модели:", accuracy_score(target_train, target_train_pred_constant))

Accuracy константной модели: 0.7935460212687936


Матрица ошибок построенной модели решающего дерева:

In [19]:
confusion_matrix(target_valid, predicted_valid)

array([[1252,  189],
       [ 192,  185]])

Позитивные ответы модель угадывает не очень хорошо, только 50%.

In [20]:
print("Полнота:", recall_score(target_valid, predicted_valid))

Полнота: 0.4907161803713528


Значение recall далеко от идеала (единицы): модель не очень хорошо ищет положительные объекты.

**Вывод:** 
- Accuracy модели: 0.7904
- Accuracy константной модели: 0.793

Accuracy модели меньше, чем случайной константной модели. F1-мера всего 0.49, auc-roc - 0.68. Модель часто ошибается при присвоении положительных ответов. Ее можно улучшить, поработав с дисбалансом классов. Судя по расчетам выше, он довольно большой.

## Борьба с дисбалансом

Для борьбы с дисбалансом я попробую **увеличить выборку**:

In [21]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    repeat = repeat
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled
    
features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

print(features_upsampled.shape) #проверка
print(target_upsampled.shape)

(8832, 11)
(8832,)


Проверю баланс классов после увеличения выборки:

In [22]:
target_upsampled.value_counts()

1    4504
0    4328
Name: exited, dtype: int64

Классы разделились почти поровну, что очень хорошо. Обучу модель и посмотрю на F1-меру и AUC-ROC:

In [23]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print('F1:', f1_score(target_valid,predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

print("AUC-ROC:", roc_auc_score(target_valid, probabilities_one_valid))

F1: 0.46195652173913043
AUC-ROC: 0.6598847322722027


Матрица ошибок модели:

In [24]:
confusion_matrix(target_valid, predicted_valid)

array([[1252,  189],
       [ 207,  170]])

Модель плохо угадывает положительные ответы, а показатель F1 ниже, чем был. Похоже, этот способ не очень сработал.

Попробую **сбалансировать классы настройкой** при обучении модели:

In [25]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)

f1_score(target_valid, predicted_valid)

0.47248322147651006

Модель все еще работает не очень хорошо. Попробую **уменьшить выборку**:

In [26]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.4)

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.4987775061124694


Проверю баланс классов после уменьшения:

In [27]:
target_downsampled.value_counts()

0    1731
1    1126
Name: exited, dtype: int64

С этим можно работать, так как F1-меры больше у меня не было. Теперь буду искать модель и перебирать гиперпараметры:

### DecisionTreeClassifier

In [28]:
best_model = None
best_result = 0

for depth in range(1, 11):
    for crit in ['gini', 'entropy']:
        for spl in ['best', 'random']:
            model = DecisionTreeClassifier(random_state=12345, max_depth=depth, criterion=crit, splitter=spl)
            model.fit(features_downsampled, target_downsampled)
            predicted_valid = model.predict(features_valid)
            result = f1_score(target_valid, predicted_valid)
            if result > best_result:
                best_model = model
                best_result = result
                d = depth
                c = crit
                s = spl

print("F1-мера наилучшей модели на валидационной выборке:", best_result, "; max_depth:", d, \
      "; criterion:", c, "; splitter:", s)

F1-мера наилучшей модели на валидационной выборке: 0.6004618937644342 ; max_depth: 5 ; criterion: entropy ; splitter: random


### RandomForestClassifier

In [29]:
best_model = None
best_result = 0

for est in range(5, 201, 10):
    for md in range(1,21):
        for mf in ['log2', 'sqrt']:
            model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=md, max_features=mf)
            model.fit(features_downsampled, target_downsampled)
            predicted_valid = model.predict(features_valid)
            result = f1_score(target_valid, predicted_valid)
            if result > best_result:
                best_model = model
                best_result = result
                estim = est
                mdt = md
                mfr = mf

print("F1-мера наилучшей модели на валидационной выборке:", best_result, "; n_estimators:", estim, "; max_depth:", \
      
      mdt, "; max_features:", mfr)

F1-мера наилучшей модели на валидационной выборке: 0.6393034825870647 ; n_estimators: 65 ; max_depth: 9 ; max_features: log2


### LogisticRegression

In [30]:
best_model = None
best_result = 0

for max_it in range(100, 1101, 100):
    for p in ['l2', 'none']:
        model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=max_it, penalty=p)
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        result = f1_score(target_valid, predicted_valid)
        if result > best_result:
            best_model = model
            best_result = result
            maxit = max_it
            pen=p
            

print("F1-мера наилучшей модели на валидационной выборке:", best_result, "; max_iter:", maxit, "; penalty:", pen)

F1-мера наилучшей модели на валидационной выборке: 0.4987775061124694 ; max_iter: 100 ; penalty: l2


**Вывод:** Избавиться от дисбаланса получилось с помощью уменьшения выборки. Был сделан перебор моделей и гиперпараметров для наилучшего результата. Лучший результат показала модель RandomForestClassifier. F1-мера = 0.639.

## Тестирование модели

Так как я выбрала модель RandomForestClassifier, протестирую ее еще раз на тестовой выборке и проверю на адекватность.

In [31]:
model = RandomForestClassifier(random_state=12345, n_estimators=65, max_depth=9, max_features='log2')
model.fit(features_downsampled, target_downsampled)
predicted_test = pd.Series(model.predict(features_test))
print("F1-мера:", f1_score(target_test, predicted_test))
print("Accuracy:", accuracy_score(target_test, predicted_test))

F1-мера: 0.6215864759427828
Accuracy: 0.840021990104453


In [32]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

print("AUC-ROC:", roc_auc_score(target_test, probabilities_one_test))

AUC-ROC: 0.8601426830309664


Баланс классов в тестовой выборке:

In [33]:
target_test.value_counts(normalize=True)

0    0.807037
1    0.192963
Name: exited, dtype: float64

Матрица ошибок:

In [34]:
confusion_matrix(target_test, predicted_test)

array([[1289,  179],
       [ 112,  239]])

Константная модель, которая ставит нули:

In [35]:
target_test_pred_constant = pd.Series(pd.Series(0, index=target_test.index))

print("Accuracy константной модели:", accuracy_score(target_test, target_test_pred_constant))

Accuracy константной модели: 0.8070368334249588


**Вывод:** F1-мера равна 0.622, AUC-ROC: 0.860, угадывать положительные ответы модель стала лучше. Тестирование проведено успешно, модель прошла проверку на адекватность.

## **Общий вывод:**

В рамках работы над задачей была выполнена **предобработка данных:**
- изменены названия ячеек,
- сделана проверка на дубли,
- удалены пропуски,
- изменены типы данных в некоторых столбцах.

**В рамках исследования задачи** для лучшего обучения модели были применены:
- прямое кодирование для перевода категориальных признаков в численные,
- разбивка данных на выборки: обучающую, тестовую и валидационную,
- удалены некоторые столбцы, которые никак не влияли на достижение цели,
- проведена стандартизация численных данных
- обучена модель без учета баланса классов, рассчитаны метрики по ней,
- определен дисбаланс классов.

Выявлено, что модель, обученная без учета дисбаланса классов, не очень хорошо справляется с поставленной задачей.

**В рамках борбьбы с дисбалансом:**
- протестировано увеличение выборки,
- протестировано уменьшение выборки,
- протестирована балансировка классов при обучении модели.

Выявлено, что лучший результат показало уменьшение выборки, именно новые полученные данные дальше использовались для поиска модели.

При поиске моделей были использованы с перебором гиперпараметров:
- DecisionTreeClassifier
- RandomForestClassifier
- LogisticRegression.

Значения F1-меры, полученные при тестировании на валидационной выборке:<br>
DecisionTreeClassifier: **0.600**<br>
RandomForestClassifier: **0.639**<br>
LogisticRegression: **0.498**<br>

По результатам тестирования на валидационной выборке была выбрана модель **RandomForestClassifier.**

**В рамках итогового тестирования** была проверена модель **RandomForestClassifier** с такими гиперпараметрами: 
- n_estimators: 65 
- max_depth: 9 
- max_features: log2

Рассчитаны метрики этой модели: <br>

F1-мера: 0.622 <br>
Accuracy: 0.840 <br>
AUC-ROC: 0.860 <br>

Также для проверки адекватности была использована константная модель, которая автоматически поставила нули всем предсказаниям. Accuracy этой модели получилась равной 0.807, что меньше, чем у выбранной мной обученной и протестированной модели.

**Цель задачи:** построить модель с предельно большим значением F1-меры. Нужно довести метрику до 0.59. **Эта цель выполнена, так как F1-мера равна 0.622.**